<a href="https://colab.research.google.com/github/VishanOberoi/FineTuningForTheGPUPoor/blob/main/Chainlit_Frontend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This incorporates all of the previous steps into a frontend.

#Using the open source project: Chainlit.

In [ ]:
!pip install langchain langchain_experimental openai chainlit ctransformers langchain_community pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 5.4 MB/s eta

In [ ]:
%%writefile app1.py

import chainlit as cl
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



# Configuration for the language model
config = {
    'gpu_layers': 100,
    'max_new_tokens': 500,
    'context_length': 4000,
    'temperature': 0.2,
    'top_k': 40,
    'top_p': 0.6
}

llm = CTransformers(
    model = "vishanoberoi/Llama-2-7b-chat-hf-finedtuned-to-GGUF",
    model_file="finetuned.gguf",
    model_type="llama",
    callbacks=[StreamingStdOutCallbackHandler()],
    config = config
    )


# Define the prompt template
template = '''
[INST]
<<SYS>>
As a chatbot for the Company "Maitri Lab Grown Diamonds," your primary focus is to provide concise, accurate, and helpful information about the company and the diamond industry. Engage in discussions related to diamonds, including sourcing, types, and care, as well as information specific to Maitri Diamonds. When asked about unrelated topics, politely redirect the conversation to the company, your areas of expertise, or inform the user that the topic falls outside your scope. Maintain a professional and neutral tone, respecting all viewpoints and ensuring accuracy in your responses.
Be humble.
Respond to the following question as a chatbot for the company named "MAITRI LAB GROWN DIAMONDS."
<</SYS>>

{question}[/INST]'''

# Define the chat start function
@cl.on_chat_start
def on_chat_start():
    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

    cl.user_session.set("llm_chain", llm_chain)


# Define the message function
@cl.on_message
async def on_message(message: cl.Message):
    llm_chain = cl.user_session.get("llm_chain")


    cb = cl.AsyncLangchainCallbackHandler(
        stream_final_answer=True
    )
    res = await llm_chain.acall(message.content, callbacks = [cb])


    await cl.Message(content=res['text']).send()

# Ensure you choose an available port or manage the port conflict.




Writing app1.py


In [ ]:
!ngrok config add-authtoken  #replace with your token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://2209-34-23-118-112.ngrok-free.app


In [ ]:
!chainlit run app1.py

Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/lib/python3.10/linecache.py", line 46, in getlines
    return updatecache(filename, module_globals)
  File "/usr/lib/python3.10/linecache.py", line 136, in updatecache
    with tokenize.open(fullname) as fp:
  File "/usr/lib/python3.10/tokenize.py", line 396, in open
    encoding, lines = detect_encoding(buffer.readline)
  File "/usr/lib/python3.10/tokenize.py", line 365, in detect_encoding
    first = read_or_stop()
  File "/usr/lib/python3.10/tokenize.py", line 323, in read_or_stop
    return readline()
KeyboardInterrupt

Original exception was:
Traceback (most recent call last):
  File "/usr/local/bin/chainlit", line 5, in <module>
    from chainlit.cli import cli
  File "/usr/local/lib/python3.10/dist-packages/chainlit/__init__.py", line 23, in <module>
    import chainlit.input_widget as input_widget
  File "/usr/local/lib/python3.10/dist-packages/chainlit/input_widget.py", line 5, in <module>
    from chainl

In [ ]:
ngrok.kill()